# HTB ReAct Agent
- **Author**: Ella Duffy
- **Version**: 1.0.0
- **Description**: Implements a ReAct (Reasoning + Action + Observation) agent for HTB pentesting using LangGraph. This agent uses existing tools from `ssh_command_executor.py` to perform port scanning, user enumeration, and SSH port detection, with plans for brute-forcing and input sanitization.

## Objective
- Automate HTB attack chaining with a stateful, reasoning-based agent.
- Leverage `scan_ports`, `enumerate_users`, and `detect_ssh_port` and `ssh_execute_command` tools.
- Prepare for future enhancements like brute-forcing and safety checks.

## Prerequisites
- Parrot OS VM (e.g., `10.0.0.215`) with HTB VPN active.
- `ssh_command_executor.py` in the working directory.
- OpenAI API key set in `~/Ella_AI/.env` as `OPENAI_API_KEY`.

## Overview of ReAct Framework Implementation
This HTB ReAct agent is designed based on the **ReAct (Reasoning + Acting)** framework, which synergizes step-by-step reasoning with action execution to solve complex tasks. Here's how it aligns:

- **Reasoning**: The agent uses a custom prompt template that instructs the language model (e.g., `gpt-4o-mini`) to think step-by-step, analyzing the task, current state, previous results, and memory to decide the next action. The "Let's think step by step" directive encourages deliberate planning.

- **Acting**: The agent leverages a set of tools (`scan_ports`, `enumerate_users`, `detect_ssh_port`, `ssh_execute_command`) integrated via LangGraph's `create_react_agent`. These tools allow the agent to interact with the HTB environment (e.g., scanning ports or executing SSH commands) based on its reasoning.

- **Observation**: Results from tool executions are stored in the `AgentState` (`result` field) and memory (`ChatMessageHistory`), enabling the agent to observe outcomes and refine its approach. The state graph's transitions (e.g., from `ports_scanned` to `enumerate_users`) facilitate this iterative process.

- **Iteration**: The agent repeats the reason-act-observe cycle, guided by the `next_step` function, until the task (e.g., reconnaissance and command execution) is complete, aligning with ReAct's iterative nature.

- **Implementation Approach**: Initially, the agent uses a hardcoded `StateGraph` with predefined nodes and transitions to ensure a reliable workflow for pentesting tasks. However, the development is moving toward a more dynamic approach, where the language model will drive tool selection and action sequences autonomously using a `ToolExecutor`, reducing reliance on fixed graph structures for greater adaptability.

This hybrid approach balances reliability with flexibility, with future enhancements planned to fully embrace the dynamic ReAct paradigm.

![ReAct Framework](images/react.png)

In [ ]:
#Imports and setup

import logging
import os
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.agents import Tool
from langchain_core.runnables import RunnableLambda
from ssh_command_executor import Tools #my defined Tools class
from typing import TypedDict, Annotated
import operator

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('logs/react_attack.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

logger.info("Loading HTB ReAct Agent")

# Load API key from environment
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in environment. Set it in .env or export it manually.")
logger.info("API key loaded successfully")

# Initialize LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, api_key=api_key)

# Initialize memory
memory = ChatMessageHistory()

# Initialize my defined Tools that are imported from ssh_command_executor
tools = Tools(memory=memory)

2025-07-08 21:37:18,258 - INFO - Loading SSH Command Executor
2025-07-08 21:37:18,275 - INFO - Loading HTB ReAct Agent
2025-07-08 21:37:18,276 - INFO - API key loaded successfully


In [2]:
#Define tools for the agent
tools_list = [
    Tool(
        name="ssh_execute_command",
        func=lambda host, username, command: tools.ssh_command_executor(host, username, command),
        description="Execute a custom SSH command on the target HTB machine. Args: host (str), username (str), command (str)"
    ),
    Tool(
        name="scan_ports",
        func=lambda host, username, target: tools.scan_ports(host, username, target),
        description="Scan open ports on the target HTB machine and return a dictionary of port:service pairs. Args: host (str), username (str), target (str)"
    ),
    Tool(
        name="enumerate_users",
        func=lambda host, username, target: tools.enumerate_users(host, username, target),
        description="Enumerate SSH users on the target HTB machine if finger service (port 79) is available. Args: host (str), username (str), target (str)"
    ),
    Tool(
        name="detect_ssh_port",
        func=lambda host, username, target: tools.detect_ssh_port(host, username, target),
        description="Detect the SSH port on the target HTB machine using memory-based analysis. Args: host (str), username (str), target (str)"
    )
]

In [3]:
# Define the Agent State
class AgentState(TypedDict):
    task: str
    memory: str
    result: Annotated[list, operator.add]
    current_step: str
    host: str
    username: str
    target: str

In [4]:
# Define the ReAct prompot template
prompt = ChatPromptTemplate.from_template(
    """You are an AI assistance for Hack The Box pentesting. Use the provided tools to perform taks on the HTB target.
    Follow this process:
    1. Reason about the task and decide which tool(s) to use based on the current state and previous results.
    2. Take action by calling the appropriate tool with the correct arguments.
    3. Observe the result and decide the next step, repeating if necessary, or use ssh_execute_command for custom actions.
    4. Return the final result when the task is complete.

    Tools available: {tool_names}

    Task: {task}

    Previous conversation: {memory}

    Current state: {current_step}
    Previous results: {result}

    Reasoning: Let's think step by step:
    
    """
)

In [5]:
# Create the ReAct agent
agent = create_react_agent(
    model=llm, 
    tools=tools_list, 
    prompt=prompt
    )

In [6]:
# Build the state graph

def scan_ports_node(state: AgentState):
    logger.info(f"Executing scan_ports on {state['target']}")
    result = tools.scan_ports(state['host'], state['username'], state['target']) #returns dictionary of port:service pairs
    return {"result": [result], "current_step": "ports_scanned"}

def enumerate_users_node(state: AgentState):
    logger.info(f"Executing enumerate_users on {state['target']}")
    if "79" in state["result"][-1]:
        result = tools.enumerate_users(state['host'], state['username'], state['target'])
        return {"result": state["result"] + [result], "current_step": "users_enumerated"}
    return {"result": state["result"], "current_step": "users_skipped"}

def detect_ssh_port_node(state: AgentState):
    logger.info(f"Executing detect_ssh_port on {state['target']}")
    result = tools.detect_ssh_port(state['host'], state['username'], state['target'])
    return {"result": state["result"] + [result], "current_step": "ssh_detected"}

def execute_command_node(state: AgentState):
    logger.info(f"Executing ssh_execute_command on {state['target']}")
    # Allow LLM to decide the command via reasoning
    result = tools.ssh_command_executor(state['host'], state['username'], "whoami") # Default command, LLM can override
    return {"result": state["result"] + [result], "current_step": "command_executed"}

# Define conditional transitions
def next_step(state: AgentState):
    if state["current_step"] == "command_executed":
        return END
    if state["current_step"] == "ports_scanned":
        return "enumerate_users"
    if state["current_step"] == "users_enumerated" or state["current_step"] == "users_skipped":
        return "detect_ssh_port"
    if state["current_step"] == "ssh_detected":
        return "execute_command"
    return "scan_ports"  # Fallback to start

graph = StateGraph(AgentState)
graph.add_node("scan_ports", scan_ports_node)
graph.add_node("enumerate_users", enumerate_users_node)
graph.add_node("detect_ssh_port", detect_ssh_port_node)
graph.add_node("execute_command", execute_command_node)

graph.set_entry_point("scan_ports")
graph.add_conditional_edges("scan_ports", next_step)
graph.add_conditional_edges("enumerate_users", next_step)
graph.add_conditional_edges("detect_ssh_port", next_step)
graph.add_conditional_edges("execute_command", next_step)

app = graph.compile()


In [ ]:
# Define a simple task function
def run_react_agent(host, username, target):
    logger.info(f"Starting ReAct agent on {target} via {host}")
    initial_state = {
        "task": f"Perform reconnaissance on the HTB target {target} by scanning ports, enumerating users if possible, detecting the SSH port, and running a custom command. Use ssh_execute_command to explore further if needed.",
        "memory": "\n".join(msg["content"] for msg in memory.messages if "content" in msg), #ensure we are working with human-readable string representation
        "result": [],
        "current_step": "start",
        "host": host,
        "username": username,
        "target": target
    }
    result = app.invoke(initial_state)
    logger.info(f"ReAct agent completed: {result['result']}")
    return result['result']

In [ ]:
# Run the Agent
if __name__ == "__main__":
    host = "10.0.0.215"
    username = "user"
    target = "10.10.10.76"
    result = run_react_agent(host, username, target)
    print("Final result:", result)

    # Debug memory
    print("Memory content:", "\n".join(msg["content"] for msg in memory.messages if "content" in msg)) 